<a href="https://colab.research.google.com/github/ladiaJeong/-01-classification/blob/master/%5B13%5D_%EC%98%81%ED%99%94_%EC%B6%94%EC%B2%9C_%EC%8B%9C%EC%8A%A4%ED%85%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1) 데이터 준비와 전처리

In [ ]:
import numpy as np
import scipy
import implicit

import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(10)

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [ ]:
ratings = ratings[ratings['ratings']>=3] #3점 미만 제외
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [ ]:
ratings['counts'].describe()

count    836478.000000
mean          3.958293
std           0.762280
min           3.000000
25%           3.000000
50%           4.000000
75%           5.000000
max           5.000000
Name: counts, dtype: float64

In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#### 2) 분석해 봅시다.

In [ ]:
using_cols = ['user_id','movie_id','counts']
data = ratings[using_cols]

In [ ]:
data.head(10)

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [ ]:
data['user_id'].nunique()

6039

In [ ]:
data[['movie_id']].nunique()

movie_id    3628
dtype: int64

In [ ]:
movies_idx = movies[['movie_id','title']].copy() #merge를 위한 df
movie_count = data['counts'].groupby(data['movie_id']).mean()
movie_count = pd.merge(movie_count, movies_idx, left_on = 'movie_id', right_on = 'movie_id', how='left')
movie_count.sort_values(by = 'counts',ascending=False).head(10)


,movie_id,counts,title
1616,1830,5.0,Follow the Bitch (1998)
3296,3607,5.0,One Little Indian (1973)
3481,3800,5.0,Criminal Lovers (Les Amants Criminels) (1999)
2993,3280,5.0,"Baby, The (1973)"
904,989,5.0,Schlafes Bruder (Brother of Sleep) (1995)
1235,1360,5.0,Identification of a Woman (Identificazione di ...
3341,3656,5.0,Lured (1947)
646,687,5.0,Country Life (1994)
3559,3881,5.0,Bittersweet Motel (2000)
729,787,5.0,"Gate of Heavenly Peace, The (1995)"


In [ ]:
count_movie = data['movie_id'].value_counts().to_frame()
count_movie.rename(columns={'movie_id':'views'},inplace=True)
count_movie.index.name= 'movie_id'
count_movie.reset_index(inplace=True)

In [ ]:
count_movie = pd.merge(movie_count, count_movie, left_on = 'movie_id', right_on = 'movie_id', how='left')
count_movie.loc[count_movie['views']  > 1000].sort_values(by = 'counts',ascending=False).head(60) #views를 추가해서 보니 더 정확한 리뷰가 될것 같습니다.


,movie_id,counts,title,views
785,858,4.598523,"Godfather, The (1972)",2167
305,318,4.596627,"Shawshank Redemption, The (1994)",2194
49,50,4.577982,"Usual Suspects, The (1995)",1744
509,527,4.571112,Schindler's List (1993),2257
698,750,4.539334,Dr. Strangelove or: How I Learned to Stop Worr...,1322
249,260,4.528522,Star Wars: Episode IV - A New Hope (1977),2910
1082,1198,4.520421,Raiders of the Lost Ark (1981),2473
823,904,4.505780,Rear Window (1954),1038
2600,2858,4.499844,American Beauty (1999),3211
842,923,4.493010,Citizen Kane (1941),1073


#### 3) 내가 선호하는 영화를 6가지 골라서 ratings에 추가해 줍시다.

In [ ]:
my_favorite = [50,2762,2324,110,1225,356] 

In [ ]:
my_playlist = pd.DataFrame({'user_id': ['hyo']*6, 'movie_id': my_favorite, 'counts':[5]*6})
my_playlist

,user_id,movie_id,counts
0,hyo,50,5
1,hyo,2762,5
2,hyo,2324,5
3,hyo,110,5
4,hyo,1225,5
5,hyo,356,5


In [ ]:
if not data.isin({'user_id':['hyo']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)     

,user_id,movie_id,counts
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,hyo,50,5
1,hyo,2762,5
2,hyo,2324,5
3,hyo,110,5
4,hyo,1225,5
5,hyo,356,5


In [ ]:
data = pd.merge(data, movies, left_on = 'movie_id',right_on = 'movie_id', how='left')
len(data['movie_id'].unique())
len(data['title'].unique()) # 중복된 타이틀이 있을 수 있어 위 unique 숫자와 일치

data['title'] = data['title'].str.lower()

In [ ]:
my_movies = list(data.loc[data['movie_id'].isin(my_favorite)].title.unique())
my_movies

['sixth sense, the (1999)',
 'braveheart (1995)',
 'amadeus (1984)',
 'forrest gump (1994)',
 'usual suspects, the (1995)',
 'life is beautiful (la vita è bella) (1997)']

In [ ]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [ ]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['hyo'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(movie_to_idx['forrest gump (1994)'])

6039
160


In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()

In [ ]:
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

user_id column indexing Fail!!


In [ ]:
# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()

In [ ]:
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

movie column indexing OK!!


#### 4) CSR matrix를 직접 만들어 봅시다.

In [ ]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836484 stored elements in Compressed Sparse Row format>

#### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836484 stored elements in Compressed Sparse Column format>

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
hyo, forrestgump = user_to_idx['hyo'], movie_to_idx['forrest gump (1994)']
hyo_vector, forrestgump_vector = als_model.user_factors[hyo], als_model.item_factors[forrestgump]

print('슝=3')

슝=3


In [ ]:
hyo_vector

array([-0.3501469 ,  0.4132461 , -1.1299404 ,  0.19178091,  0.21520098,
        0.39348274,  0.03238026, -0.21933205, -1.2977104 ,  0.05814508,
        1.5397824 ,  0.12843435, -1.4572076 , -0.22752616, -0.05666407,
        0.21833563, -0.44869286, -0.07639629,  0.59494346, -0.17081118,
        1.1302427 , -0.03457879, -0.55730444,  0.06649344, -0.20409325,
        0.21938124, -1.068419  ,  0.22793597,  0.48972377, -0.2573422 ,
       -0.8661196 ,  0.43176305,  0.6844863 , -0.60856485,  0.75465584,
       -0.08127493, -1.1966656 , -0.30432191, -0.15485497, -0.3621141 ,
       -0.48333675,  0.09180984, -0.12799582, -0.09282832, -0.18076104,
        0.14952974,  0.14968918, -0.08169127,  0.02196236,  0.8188432 ,
       -0.64561087,  0.5022313 , -0.40755114,  0.20148496, -0.20946355,
        0.42670035, -0.7882224 , -0.81477773,  1.6179616 , -0.618698  ,
       -0.62493086,  0.07985348,  0.95944804,  0.01114063, -0.00254874,
        0.25313297,  0.5726736 , -0.10950752,  0.28857937, -0.77

In [ ]:
forrestgump_vector

array([ 1.06155565e-02, -2.01508794e-02,  2.27088016e-03, -1.97601430e-02,
        2.28382666e-02,  9.83095728e-03, -3.41138919e-03,  2.14722641e-02,
       -3.79212238e-02,  2.89552491e-02,  1.94706209e-02,  5.78152731e-06,
        1.95304845e-02, -5.17141726e-03, -2.15715263e-02,  4.74553835e-03,
        4.95541655e-03, -1.83936879e-02,  2.20593065e-02, -2.46336348e-02,
        3.22183557e-02,  4.56100516e-02, -2.10720636e-02, -2.29106378e-03,
       -1.38574059e-03, -1.10409195e-02,  1.57769043e-02,  3.01788617e-02,
        3.82251255e-02, -5.77787124e-03, -4.90609463e-03,  2.53726598e-02,
        3.14064845e-02,  7.78180268e-03,  4.35305312e-02,  1.29095633e-02,
        4.06013988e-03,  1.36425216e-02,  5.66225592e-03,  2.88059879e-02,
       -1.86484549e-02,  1.91010144e-02,  1.02615999e-02, -2.28179432e-02,
       -8.09087139e-03,  4.56084982e-02,  2.13270336e-02,  1.36449318e-02,
       -2.95675956e-02,  1.53212035e-02, -4.08911109e-02, -1.62343159e-02,
        2.27853134e-02,  

In [ ]:
np.dot(hyo_vector,forrestgump_vector)

0.53782713

In [ ]:
amadeus = movie_to_idx['amadeus (1984)']
amadeus_vector = als_model.item_factors[amadeus]
np.dot(hyo_vector, amadeus_vector)

0.35659668

In [ ]:
my_movies[3]

'forrest gump (1994)'

In [ ]:
favorite_movie = my_movies[3] 
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=10)
similar_movie

[(160, 1.0),
 (110, 0.56265396),
 (666, 0.42141473),
 (613, 0.4212299),
 (336, 0.41003314),
 (20, 0.3994214),
 (204, 0.38595238),
 (109, 0.3810966),
 (154, 0.37912607),
 (64, 0.3728477)]

In [ ]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movies = {v:k for k,v in movie_to_idx.items()}
[idx_to_movies[i[0]] for i in similar_movie]

['forrest gump (1994)',
 'groundhog day (1993)',
 'sleepless in seattle (1993)',
 'four weddings and a funeral (1994)',
 'pretty woman (1990)',
 'pleasantville (1998)',
 'clueless (1995)',
 'doctor zhivago (1965)',
 'as good as it gets (1997)',
 'star wars: episode vi - return of the jedi (1983)']

In [ ]:
def get_similar_artist(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movies[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [ ]:
get_similar_artist('amadeus (1984)')

['amadeus (1984)',
 'gandhi (1982)',
 'chariots of fire (1981)',
 'right stuff, the (1983)',
 'dangerous liaisons (1988)',
 'dead poets society (1989)',
 'to kill a mockingbird (1962)',
 'driving miss daisy (1989)',
 'raging bull (1980)',
 'my left foot (1989)']

In [ ]:
get_similar_artist('usual suspects, the (1995)')

['usual suspects, the (1995)',
 'reservoir dogs (1992)',
 'l.a. confidential (1997)',
 'pulp fiction (1994)',
 'seven (se7en) (1995)',
 'fargo (1996)',
 'goodfellas (1990)',
 'silence of the lambs, the (1991)',
 'simple plan, a (1998)',
 'sixth sense, the (1999)']

#### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [ ]:
user = user_to_idx['hyo']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(121, 0.5357449),
 (23, 0.5167656),
 (157, 0.46951258),
 (51, 0.4438395),
 (48, 0.43482274),
 (248, 0.4058426),
 (224, 0.36947858),
 (99, 0.36717725),
 (126, 0.32531482),
 (222, 0.31867307),
 (220, 0.30961666),
 (60, 0.29672328),
 (141, 0.28840438),
 (154, 0.28230426),
 (110, 0.26459354),
 (107, 0.25542095),
 (81, 0.24814945),
 (75, 0.23293635),
 (289, 0.23215273),
 (269, 0.22860684)]

In [ ]:
[idx_to_movies[i[0]] for i in movie_recommended]

['silence of the lambs, the (1991)',
 "schindler's list (1993)",
 'shawshank redemption, the (1994)',
 'fargo (1996)',
 'saving private ryan (1998)',
 'good will hunting (1997)',
 'l.a. confidential (1997)',
 'american beauty (1999)',
 'shakespeare in love (1998)',
 'pulp fiction (1994)',
 'seven (se7en) (1995)',
 'star wars: episode i - the phantom menace (1999)',
 'fugitive, the (1993)',
 'as good as it gets (1997)',
 'groundhog day (1993)',
 'jurassic park (1993)',
 'green mile, the (1999)',
 'hunt for red october, the (1990)',
 'reservoir dogs (1992)',
 'goodfellas (1990)']

In [ ]:
goodwill = movie_to_idx['good will hunting (1997)']
explain = als_model.explain(user, csr_data, itemid=goodwill)

In [ ]:
[(idx_to_movies[i[0]], i[1]) for i in explain[1]]

[('braveheart (1995)', 0.09894960678332017),
 ('sixth sense, the (1999)', 0.09053970407409861),
 ('life is beautiful (la vita è bella) (1997)', 0.07135851174256919),
 ('usual suspects, the (1995)', 0.06363741601750747),
 ('amadeus (1984)', 0.04475210488842651),
 ('forrest gump (1994)', 0.02889390522436016)]

### 회고
1. 추천 시스템을 만들어 보았다. 내가 좋아하는 영화를 기본 데이터로 비슷한 영화를 추천하는것으로 콘텐츠 기반 필터링을 해 보았다. 
2. 결과 값으로 장르와 관련이 있는 영화들을 추천하는 결과의 양상을 보이며, 영화가 개봉한 시기하고도 관련이 있는듯하다. 추천이라는 것이 상당히 많은 부분을 고려해야하는 항목이기 때문에 정확한 예측이라고 볼 수 는 없지만, 벡터값에 따라 더 비슷한 장르와 시기의 영화가 추천이 되기는 한다. 행렬의 곱의 결과 수치가 높을 수록 더 좋은 추천 결과라고 볼 수 있다. 
3. 내가 정하는 6가지 영화가 어느 정도 관련성이 있어야 보다 정확한 추천 결과를 볼 수 있을 것 같다. 